## 📝 The Smart Recipe Planner: Setup

**Project:** The Smart Recipe Planner
**Track:** Concierge Agents
**Subtitle:** A Sequential Agent System for Recipe and Grocery Planning

This notebook demonstrates a complete, sequential multi-agent system. The goal is to build a "Concierge Agent" that can take a user's meal idea and turn it into an actionable plan.

The first step is to set up our environment. The code cell below will:
1.  Install the `google-generativeai` library.
2.  Import necessary libraries.
3.  Securely configure the `GOOGLE_API_KEY` from Kaggle Secrets.
4.  Define the model, we'll use for our agents (`gemini-2.5-flash-lite`).

In [1]:
import os
from google import genai  # <-- This is the NEW SDK
from kaggle_secrets import UserSecretsClient

# --- 1. Setup & API Key Configuration ---
# Use your code to set the API key as an environment variable
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

# This is the model string for the new SDK
MODEL_NAME = "models/gemini-2.5-flash-lite"

# Create a single client to be used by all agents
# genai.Client() will automatically find the GOOGLE_API_KEY in the environment
client = genai.Client()

print("--- Project: The Smart Recipe Planner ---")

✅ Gemini API key setup complete.
--- Project: The Smart Recipe Planner ---


## Step 1: Define the User's Request

This cell defines the initial human input. This natural language string is the single trigger that will start our entire multi-agent workflow.

In [2]:
# --- 2. Define the User's Request ---
user_request = "I want to make a simple vegetarian pasta dish for tonight."
print(f"User Request: {user_request}\n")

User Request: I want to make a simple vegetarian pasta dish for tonight.



## 🤖 Step 2: Agent 1 - The `RecipeAgent`

This is the first agent in our **sequential multi-agent system**.

* **Key Concept:** **Agent powered by an LLM**
* **How it works:** This `RecipeAgent` takes the `user_request`. Its job is to use the LLM's reasoning to understand the user's intent (e.g., "simple," "vegetarian") and find one matching recipe. Its prompt is heavily engineered to return *only* the recipe name and a clean ingredient list.

The output of this agent is critical, as it forms the "state" or "memory" that will be passed to the next agent.

In [3]:
# --- 3. Agent 1: The RecipeAgent ---
print("--- Agent 1 (RecipeAgent) is thinking... ---")
# We will use the client created in Cell 1

recipe_prompt = f"""
You are a recipe assistant. The user request: '{user_request}'.
Find one simple, popular recipe for this.
Your response MUST be only the recipe name and its ingredients (all the necessary ingredients) as a clean list.
Do not add any other text, chatter, or formatting.
"""

try:
    # Use the new client.models.generate_content method
    response_agent1 = client.models.generate_content(
        model=MODEL_NAME,
        contents=recipe_prompt
        # No tool config needed for this agent
    )
    recipe_output = response_agent1.text
    print(f"Agent 1 Output (Recipe):\n{recipe_output}\n")
    
    # --- 4. Session & State Management ---
    ingredient_list = recipe_output

except Exception as e:
    print(f"Agent 1 failed: {e}")
    ingredient_list = None

--- Agent 1 (RecipeAgent) is thinking... ---
Agent 1 Output (Recipe):
One-Pot Lemon Herb Pasta

12 ounces spaghetti
4 cups vegetable broth
1/2 cup heavy cream
1/4 cup grated Parmesan cheese
2 tablespoons olive oil
1 tablespoon lemon juice
1 teaspoon dried oregano
1 teaspoon dried basil
1/2 teaspoon garlic powder
Salt to taste
Black pepper to taste
Fresh parsley, chopped, for garnish



## 🛒 Step 3: Agent 2 - The `ShoppingAgent` & Final Result

This is the final step in our pipeline and demonstrates three key concepts at once:

1.  **Sequential Agents:** This `ShoppingAgent` **cannot** run without the output (the `ingredient_list`) from Agent 1. This dependency clearly shows a sequential agent architecture.
2.  **Sessions & State Management:** The `ingredient_list` variable is the "session state." By passing it from Agent 1 to Agent 2, we are demonstrating simple but effective session management.
3.  **Tools (built-in tools):** This agent is equipped with the **`GoogleSearchRetriever`** tool. The LLM is instructed to use this tool to find a real-world grocery store, making the plan actionable for the user.

The cell will then print the final, consolidated plan, completing the "idea-to-action" workflow.

In [4]:
# --- 5. Agent 2: The ShoppingAgent (with a Tool) ---
if ingredient_list: 
    print("--- Agent 2 (ShoppingAgent) is thinking... ---")
    
    # Import the NEW SDK components for tools
    from google.genai.types import (
        GenerateContentConfig,
        GoogleSearch,
        Tool,
    )

    # Define the tool for the NEW SDK
    search_tool = Tool(google_search=GoogleSearch())

    # Define the config for the NEW SDK
    tool_config = GenerateContentConfig(
        tools=[search_tool],
    )

    shopping_prompt = f"""
    You are a shopping assistant. A user has the following recipe and ingredients:
    {ingredient_list}

    Use your Google Search tool to find ONE major grocery store (like Walmart, Kroger, Safeway, etc.)
    near 'Mountain View, CA' where they can buy these items.

    Respond with only the store name and its full address.
    """

    try:
        # Call the model using the NEW SDK's client
        response_agent2 = client.models.generate_content(
            model=MODEL_NAME,
            contents=shopping_prompt,
            config=tool_config,
        )
        shopping_output = response_agent2.text
        print(f"Agent 2 Output (Shopping):\n{shopping_output}\n")
        
        # --- 6. Final Result ---
        print("--- Sequential Agent Run Complete ---")
        print(f"Final Plan: To make your dish, you can buy ingredients at: {shopping_output}")

    except Exception as e:
        print(f"Agent 2 failed: {e}")
else:
    print("Agent 2 was skipped because Agent 1 failed.")

--- Agent 2 (ShoppingAgent) is thinking... ---
Agent 2 Output (Shopping):
Safeway
645 San Antonio Rd, Mountain View, CA 94040

--- Sequential Agent Run Complete ---
Final Plan: To make your dish, you can buy ingredients at: Safeway
645 San Antonio Rd, Mountain View, CA 94040


## ✅ Project Conclusion & Key Concepts

This notebook successfully demonstrated a complete, end-to-end multi-agent system. We built a **Concierge Agent** that solves a common, real-world problem—the "dinner dilemma"—by automating the entire workflow from idea to action.

As shown in the output above, the system:
1.  Understood a vague user request ("simple vegetarian pasta").
2.  **Agent 1** (the `RecipeAgent`) found a specific recipe and extracted its ingredients.
3.  The **session state** (the ingredient list) was successfully passed to the next agent.
4.  **Agent 2** (the `ShoppingAgent`) used its **built-in tool** (Google Search) to find a real, actionable location.

### Key Concepts Demonstrated:
This project, though concise, clearly demonstrates **four** of the key concepts from the course:

* **Multi-agent system (Sequential agents):** We built a two-agent pipeline where the `ShoppingAgent` was dependent on the output from the `RecipeAgent`.
* **Agent powered by an LLM:** Both agents used the `gemini-2.5-flash-lite` model to perform reasoning tasks.
* **Tools (built-in tools):** The `ShoppingAgent` was given the `GoogleSearchRetriever` tool and successfully used it to find real-world information.
* **Sessions & Memory (Sessions & state management):** The `ingredient_list` variable acted as the simple, yet critical, session state passed between the agents.

Given the time constraints, this project focused on a clean, robust, and clear implementation of these core concepts.